In [1]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 82.8 MB/s eta 0:00:00


In [2]:
from pulp import *
import pandas as pd
from openpyxl import load_workbook, Workbook
import re
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Ejemplo Mod Distribucion

In [ ]:
# CREAMOS MODELO
mod = LpProblem("distribución", LpMinimize)


# CARGAMOS DATOS
df_nodos = pd.read_excel(r"piloto mod distribución.xlsx",sheet_name="nodos")
df_materiales = pd.read_excel(r"piloto mod distribución.xlsx",sheet_name="materiales")
df_tiempos = pd.read_excel(r"piloto mod distribución.xlsx",sheet_name="tiempos")


# CREAMOS LOS CONJUNTOS

nodos = list(df_nodos['nodo']) # origenes + destinos
nodos_origen = list(df_nodos['nodo'].iloc[0:2])
nodos_destino = list(df_nodos['nodo'].iloc[2:4])

materiales =list(df_materiales['material'].iloc[0:])

tiempos = list(df_tiempos['tiempo'].iloc[0:])


# CREAMOS LOS PARAMETROS

# Parametros origen
Capacidad_Vehiculo = dict(zip(nodos_origen,df_nodos['cap_veh'].iloc[0:2]))

oferta3 = pd.read_excel(r"piloto mod distribución.xlsx", sheet_name='oferta')
oferta2 = oferta3.values.tolist()
oferta = makeDict([nodos_origen, materiales],oferta2,0)

# Parametros destino

demanda3 = pd.read_excel(r"piloto mod distribución.xlsx", sheet_name='demanda')
demanda2 = demanda3.values.tolist()
demanda = makeDict([nodos_destino, materiales],demanda2,0)

# Parametros transporte

flete_propio3 = pd.read_excel(r"piloto mod distribución.xlsx", sheet_name='costo_origen_destino_propio')
flete_propio2 = flete_propio3.values.tolist()
flete_propio = makeDict([nodos_origen, nodos_destino],flete_propio2,0)

flete_tercero3 = pd.read_excel(r"piloto mod distribución.xlsx", sheet_name='costo_origen_destino_alquilado')
flete_tercero2 = flete_tercero3.values.tolist()
flete_tercero = makeDict([nodos_origen, nodos_destino],flete_tercero2,0)

# CREAMOS LAS VARIABLES

inv_sitio_o = LpVariable.dicts("Inventario_en_Sitio_o",[(m,o,t) for m in materiales for o in nodos_origen for t in tiempos],0, None) 
inv_sitio_d = LpVariable.dicts("Inventario_en_Sitio_d",[(m,d,t) for m in materiales for d in nodos_destino for t in tiempos],0, None) 
#inv_transito = LpVariable.dicts("Inventario_en_transito",[(m, o, d, t) for m in materiales for o in nodos_origen for d in nodos_destino for t in tiempos],0, None)

viajes_propios = LpVariable.dicts("Cantidad_de_viajes_propios",[(o, d, t) for o in nodos_origen for d in nodos_destino for t in tiempos],0, None,cat='Integer')
viajes_contratados = LpVariable.dicts("Cantidad_de_viajes_contratados",[(o, d, t) for o in nodos_origen for d in nodos_destino for t in tiempos],0, None,cat='Integer')

cantidad_material = LpVariable.dicts("Cantidad_de_material",[(m, o, d, t) for m in materiales for o in nodos_origen for d in nodos_destino for t in tiempos],0, None)

# CREAMOS FUNCIÓN OBJETIVO

mod += lpSum(viajes_propios[(o, d, t)]*flete_propio[o][d] for o in nodos_origen for d in nodos_destino for t in tiempos) + lpSum(viajes_contratados[(o, d, t)]*flete_tercero[o][d] for o in nodos_origen for d in nodos_destino for t in tiempos)

# CREAMOS RESTRICCIONES

# Cumplir la demanda

for m in materiales:
    for d in nodos_destino:
        mod += lpSum([cantidad_material[m, o, d, t] for o in nodos_origen for t in tiempos]) >= demanda[d][m]
        
# Oferta inicial

for o in nodos_origen:
    for m in materiales:
        mod += inv_sitio_o[m,o,0] == oferta[o][m]

        
## PENSARLE **        
for d in nodos_destino:
    for m in materiales:
        for t in tiempos[1:]:
            mod += inv_sitio_d[m,d,t] == inv_sitio_d[m,d,t-1] +lpSum([cantidad_material[m, o, d, t] for o in nodos_origen])\
             - lpSum([cantidad_material[m, o, d, t-1] for d in nodos_destino])
        
# PENSARLE #           
# No puedo enviar mas unidades de las que puedo ofertar

for o in nodos_origen:
    for m in materiales:
        for i in tiempos[:1]:
            mod += lpSum([cantidad_material[m, o, d, t] for d in nodos_destino for t in tiempos]) <= oferta[o][m]

# Restricción capacidad vehiculo por viaje

for o in nodos_origen:
    for d in nodos_destino:
        for t in tiempos:
            for m in materiales:
                mod += cantidad_material[m, o, d, t] <= (viajes_propios[o, d, t]+viajes_contratados[o, d, t])*Capacidad_Vehiculo[o]
    
#RESVISAR EL T PORQUE NO NECESARIAMENTE SE DEBE CUMPLIR EN CADA TIEMPO 
    
# FALTA RESTRICCIÓN DE CUANTOS VEHICULOS PROPIOS CUENTO  

# FALTA RESTRICCIÓN DE TIEMPOS: 0->1->2->3->4

for t in tiempos:
    mod += lpSum([viajes_propios[o, d, t] for o in nodos_origen for d in nodos_destino ]) +lpSum([viajes_contratados[o, d, t] for o in nodos_origen for d in nodos_destino])  >= 1
    
    
# SOLVE

mod.solve(solver = pulp.PULP_CBC_CMD(msg=True, threads=8, warmStart=True, timeLimit=260000, cuts=True, strong=True, presolve=True, gapRel=0.01))

# The status of the solution is printed to the screen
print("Status:", LpStatus[mod.status])

# Each of the variables is printed with it's resolved optimum value
for v in mod.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Costo Total = ", value(mod.objective))

FileNotFoundError: ignored

In [ ]:
nodos_origen

In [ ]:
demanda2

In [ ]:
oferta

# Ejemplo Data AC Compra de Oportunidad


In [5]:
InventarioInicial=pd.read_excel("Input - copia.xlsx",sheet_name="Inventario Inicial")

In [6]:
InventarioInicial

,1040852,1111116
0,1140000,500000


In [8]:
InventarioInicial[int(1040852)][0]

1140000

In [9]:
# Carga de Información

df_input = pd.read_excel(r"Input - copia.xlsx",sheet_name="Semanas")
df_input.Material=df_input.Material.astype('object')

# Creación de Conjuntos

semanas= list(df_input['Semanas'].unique())
materiales = list(str(i) for i in df_input.Material.unique().tolist())

# Parametros Precios
precio3=pd.read_excel(r"Input - copia.xlsx",sheet_name="Precios")
precios2 = precio3.values.tolist()
precios = makeDict([semanas, materiales],precios2,0)

# Parametros Demanda
demanda3=pd.read_excel(r"Input - copia.xlsx",sheet_name="Demanda")
demanda2 = demanda3.values.tolist()
demanda = makeDict([semanas, materiales],demanda2,0)

# # Parametros LT
# LT3=pd.read_excel(r"Input - copia.xlsx",sheet_name="LT")
# LT2 = LT3.values.tolist()
# LT = makeDict([materiales],LT2,0)
LT3=pd.read_excel("Input - copia.xlsx",sheet_name="LT")
InventarioInicial=pd.read_excel("Input - copia.xlsx",sheet_name="Inventario Inicial")

# inventarioInicial = 1140000 #kg
inventarioMaximo = 20 #25 # semanas #Buscar Equilibrio, para que no sea infactible 
inventarioMinimo = 2 # semanas #Buscar Equilibrio, para que no sea infactible 
costoKgInventario_INI = 800 # 5661 #$/kg
CostoCapitalNM = 0.22 #tasa porcentaje 
costoAlmacenamiento_valor = 270 # $/kg mes
CostoTransporte_valor = 50 # $/kg 

mod_co = LpProblem("Compra Oportunidad", LpMinimize)

#Creamos las Variables

Compra = LpVariable.dicts("Compra",[(s,m) for s in semanas for m in materiales ],0, None)
Inventario = LpVariable.dicts("Inventario",[(s,m) for s in semanas for m in materiales ],0, None) 
# CostoAlmacenamiento = LpVariable.dicts("CostoAlmacenamiento",[(s,m) for s in semanas for m in materiales ],0, None) 
# CostoInventario = LpVariable.dicts("CostoInventario",[(s,m) for s in semanas for m in materiales ],0, None) 
# CostoCapital = LpVariable.dicts("CostoCapital",[(s,m) for s in semanas for m in materiales ],0, None)
# CostoTransporte = LpVariable.dicts("CostoTransporte",[(s,m) for s in semanas for m in materiales ],0, None)
CostoTotal = LpVariable.dicts("CostoTotal",[(s,m) for s in semanas for m in materiales ],0, None)

# Función Objetivo

mod_co += lpSum(CostoTotal[(s,m)] for s in semanas for m in materiales)



# Cumplir la demanda
for m in materiales:
  for s in semanas:
        mod_co += Inventario[(s,m)]  >= demanda[s][m]

# Cumplir con politicas de Inventario

for m in materiales:
  for i,s in enumerate(semanas):
    if i > list(LT3[int(m)])[0] - 1:
        mod_co += Inventario[(s,m)]  <= demanda[s][m]*inventarioMaximo

for m in materiales:
  for i,s in enumerate(semanas):
    if i >list(LT3[int(m)])[0] - 1:
        mod_co += Inventario[(s,m)] >= demanda[s][m]*inventarioMinimo

# # Costo de los inventarios
# cont=-1
# for m in materiales:
#   for s in semanas:
#     if s == semanas[0]:
#       mod_co += CostoInventario[(s,m)]  ==(((inventarioInicial-demanda[s][m]) * costoKgInventario_INI) +(Compra[(s,m)]*precios[s][m])) 
#     else:
#       mod_co += CostoInventario[(s,m)]  == (((Inventario[(semanas[cont],m)] -demanda[s][m]) * costoKgInventario_INI) +(Compra[(s,m)]*precios[s][m])) 
#     cont=cont+1

# Costo de almacenamiento

# for m in materiales:
#   for s in semanas:
#     mod_co += CostoAlmacenamiento[(s,m)] == demanda[s][m]*inventarioMinimo* costoAlmacenamiento_valor

# Costo de Capital

# for m in materiales:
#   for s in semanas:
#     mod_co += CostoCapital[(s,m)] == CostoInventario[(s,m)] * CostoCapitalNM

# # Costo de Transporte

# for m in materiales:
#   for s in semanas:
#     mod_co += CostoTransporte[(s,m)] == Compra[(s,m)] * CostoTransporte_valor

# # Costo de Total

# for m in materiales:
#   for s in semanas:
#     mod_co += CostoTotal[(s,m)] == CostoInventario[(s,m)] + \
#                             CostoTransporte[(s,m)] + CostoAlmacenamiento[(s,m)] +CostoCapital[(s,m)] #+ Compra[(s,m)]*precios[s][m]

# Costo de Total

for m in materiales:
  for s in semanas:
    mod_co += CostoTotal[(s,m)] == Compra[(s,m)]*precios[s][m]


# Juego de Inventarios
L=2


for m in materiales:
  cont=-1
  for s in semanas:
    print(s,"contador: ",cont,"lt: ",list(LT3[int(m)])[0],"momento de compra: ",cont+1-list(LT3[int(m)])[0])
    if s == semanas[0]:
      mod_co += Inventario[(s,m)]  == InventarioInicial[int(m)][0]  - demanda[s][m]
    else:
      if cont+1-list(LT3[int(m)])[0] >=0:
        mod_co += Inventario[(s,m)]  == Inventario[(semanas[cont],m)] + Compra[(semanas[cont+1-list(LT3[int(m)])[0]],m)]  - demanda[s][m] 
      else:
        mod_co += Inventario[(s,m)]  == Inventario[(semanas[cont],m)]  - demanda[s][m] 
    cont=cont+1

for s in semanas:
    mod_co += lpSum([Compra[(s,m)] for m in materiales ])  >= 0

for s in semanas:
    mod_co += lpSum([Inventario[(s,m)] for m in materiales ])  >= 0

# SOLVE

mod_co.solve(solver = pulp.PULP_CBC_CMD(msg=True, threads=8, warmStart=True, timeLimit=260000, cuts=True, strong=True, presolve=True, gapRel=0.01))

# Estatus Solución
print("Status:", LpStatus[mod_co.status])

# EaValor Optimo por variable
for v in mod_co.variables():
    print(v.name, "=", v.varValue)
    
# Valor Función Objetivo
print("Costo Total = ", value(mod_co.objective))

# Guardando los resultados en Excel
# libroRes = Workbook()
# hojaRes = libroRes.active

# i = 1
# for v in mod_co.variables():
#     hojaRes.cell(row=i,column=1).value = v.name
#     hojaRes.cell(row=i,column=2).value = v.varValue
#     i+=1

# libroRes.save('Resultados.xlsx')

/usr/local/lib/python3.9/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


08Semana contador:  -1 lt:  2 momento de compra:  -2
09Semana contador:  0 lt:  2 momento de compra:  -1
10Semana contador:  1 lt:  2 momento de compra:  0
11Semana contador:  2 lt:  2 momento de compra:  1
12Semana contador:  3 lt:  2 momento de compra:  2
13Semana contador:  4 lt:  2 momento de compra:  3
14Semana contador:  5 lt:  2 momento de compra:  4
15Semana contador:  6 lt:  2 momento de compra:  5
16Semana contador:  7 lt:  2 momento de compra:  6
17Semana contador:  8 lt:  2 momento de compra:  7
18Semana contador:  9 lt:  2 momento de compra:  8
19Semana contador:  10 lt:  2 momento de compra:  9
20Semana contador:  11 lt:  2 momento de compra:  10
21Semana contador:  12 lt:  2 momento de compra:  11
22Semana contador:  13 lt:  2 momento de compra:  12
23Semana contador:  14 lt:  2 momento de compra:  13
24Semana contador:  15 lt:  2 momento de compra:  14
25Semana contador:  16 lt:  2 momento de compra:  15
26Semana contador:  17 lt:  2 momento de compra:  16
27Semana cont

In [ ]:
LT3=pd.read_excel("Input - copia.xlsx",sheet_name="LT")

In [ ]:
LT3

,1040852,1111116
0,2,5


In [ ]:
list(LT3[1040852])[0]

2

In [ ]:
Resultados2 = []
i=0
for v in mod_co.variables():
  variable = re.findall(r"(\w+)_",v.name)[0]
  semana = re.findall(r"(\w+)',",v.name)[0]
  codigo= re.findall(r"s*'(\d+)'",v.name)[0]
  demanda = demanda2[i][0]
  precios = precios2[i][0]
  i+=1
  if i == len(demanda2):
    i=0
  Resultados2.append({"Variable": variable, "Semana": semana, "Codigo": codigo, "Valor":v.varValue, "Demanda":demanda,"Precios":precios})

Resultado = pd.DataFrame(Resultados2)
Resultado

,Variable,Semana,Codigo,Valor,Demanda,Precios
0,Compra,08Semana,1040852,1170860.80,401629.678627,4300
1,Compra,09Semana,1040852,-0.00,369615.835670,4300
2,Compra,10Semana,1040852,259351.87,365160.044401,4300
3,Compra,11Semana,1040852,374013.44,391772.195146,4300
4,Compra,12Semana,1040852,786540.33,347632.087287,4300
...,...,...,...,...,...,...
130,Inventario,48Semana,1040852,5663089.90,410539.037835,5768
131,Inventario,49Semana,1040852,5258640.00,404449.924038,5768
132,Inventario,50Semana,1040852,4848000.30,410639.725301,5768
133,Inventario,51Semana,1040852,4439455.30,408544.954767,5768


In [ ]:
Resultado_Compras = Resultado[Resultado['Variable']=="Compra"]
Resultado_Inventario = Resultado[Resultado['Variable']=="Inventario"]
Resultado_CostoAlmacenamiento = Resultado[Resultado['Variable']=="CostoAlmacenamiento"]
Resultado_CostoInventario = Resultado[Resultado['Variable']=="CostoInventario"]
Resultado_CostoCapital = Resultado[Resultado['Variable']=="CostoCapital"]
Resultado_CostoTransporte = Resultado[Resultado['Variable']=="CostoTransporte"]
Resultado_CostoTotal = Resultado[Resultado['Variable']=="CostoTotal"]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=Resultado_Compras['Semana'], y=Resultado_Compras['Valor'], name='Compras'))

fig.add_trace(go.Scatter(x=Resultado_Inventario['Semana'], y=Resultado_Inventario['Valor'], 
                         name='Inventario', mode='lines', line=dict(color='red'), legendrank=True))

fig.add_trace(go.Scatter(x=Resultado_Compras['Semana'], y=Resultado_Compras['Demanda'], 
                         name='Demanda', mode='lines', line=dict(color='green'), legendrank=True))

fig.add_trace(go.Scatter(x=Resultado_Compras['Semana'], y=Resultado_Compras['Precios'], 
                         name='Precios', mode='lines', line=dict(color='orange'), legendrank=True), secondary_y=True)

fig.add_trace(go.Scatter(x=Resultado_CostoAlmacenamiento['Semana'], y=Resultado_CostoAlmacenamiento['Valor'], 
                         name='Costo Almacenamiento', mode='markers', line=dict(color='grey'), legendrank=True), secondary_y=True)

fig.add_trace(go.Scatter(x=Resultado_CostoInventario['Semana'], y=Resultado_CostoInventario['Valor'], 
                         name='Costo Inventario', mode='markers', line=dict(color='pink'), legendrank=True), secondary_y=True)

fig.add_trace(go.Scatter(x=Resultado_CostoCapital['Semana'], y=Resultado_CostoCapital['Valor'], 
                         name='Costo Capital', mode='markers', line=dict(color='brown'), legendrank=True), secondary_y=True)

fig.add_trace(go.Scatter(x=Resultado_CostoTransporte['Semana'], y=Resultado_CostoTransporte['Valor'], 
                         name='Costo Transporte', mode='markers', line=dict(color='gold'), legendrank=True), secondary_y=True)

fig.add_trace(go.Scatter(x=Resultado_CostoTotal['Semana'], y=Resultado_CostoTotal['Valor'], 
                         name='Costo Total', mode='markers', line=dict(color='purple'), legendrank=True), secondary_y=True)

fig.update_layout(title='Compra de Oportunidad',
                  xaxis=dict(title='Semana'),
                  yaxis=dict(title='Unidades'),
                  yaxis2=dict(title='Precios', overlaying='y', side='right'))

fig.show()


